#1 Coleta e Tratamento de Dados

##1.1 Preliminares

Truque para o google colab não desconectar:
* Colar o seguinte no console da ferramenta de desenvolvedor:
```
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,60000)
```

In [2]:
!pip install geocoder
# !pip install geopy
# !pip install foursquare

     |████████████████████████████████| 98 kB 7.2 MB/s 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import folium
import geocoder
import json
import time
import warnings

from google.colab import drive
from ast import literal_eval
from bs4 import BeautifulSoup
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from geopy.geocoders import Nominatim
from datetime import datetime, timezone, timedelta

In [4]:
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
# O caminho depende de onde o arquivo está localizado (alterar se necessário):
caminho = '/content/drive/MyDrive/Projeto_Foursquare/'

##1.2 Webscraping das informações sobre bairros de Porto Alegre

As informações sobre bairros de Porto Alegre podem ser encontradas na Wikipédia: [Lista de bairros de Porto Alegre](https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Porto_Alegre).

O webscraping neste caso pode ser feito de duas maneiras:

1. Entrar no site e acessar o item "inspecionar" do menu do botão direito, encontrar o trecho do HTML correspondente à tabela, copiar o texto e colar em um arquivo de texto com final `.html` para ser lido pelo comando `.read_html(caminho)` da biblioteca `pandas`.

2. Utilizando o comando `.get(url)` da biblioteca `requests` com a URL da página supracitada e o objeto `BeautifulSoup()` da biblioteca `bs4` para interpretar o HTML da página inteira, filtrando tabelas, linhas e conteúdos das células com o comando `find_all(...)` e organizando em um dataframe através de um laço.

###1.2.1 Direto do Arquivo HTML

In [15]:
tabela = pd.read_html(caminho + 'tabela.html')
df = tabela[0].sort_values('Bairro').reset_index().drop('index',axis=1)
# Tirando a última linha, que só mostrava os valores totais:
df = df[df['Bairro']!='TOTAL'].reset_index().drop('index',axis=1)
df

,Bairro,Data de Criação,Área,População2010,Tx Cresc Pop 91-00,Densidade,Renda média por domicílio,Nota
0,Aberta dos Morros,21 dez 2015,NaN,NaN,NaN,NaN,NaN,NaN
1,Agronomia,21 set 1976,1241 ha,12222.0,"19,24% a.a.","8,6 hab/ha","3,98 SM/mês",(1)
2,Anchieta,7 dez 1959,84 ha,203.0,"0,51% a.a.","2,4 hab/ha","8,41 SM/mês",NaN
3,Arquipélago,7 dez 1959,4718 ha,5061.0,"9,32% a.a.","1,1 hab/ha","2,96 SM/mês",NaN
4,Auxiliadora,7 dez 1959,82 ha,9985.0,"-0,25% a.a.","121,8 hab/ha","19,57 SM/mês",NaN
...,...,...,...,...,...,...,...,...
91,Vila Jardim,7 dez 1959,178 ha,14251.0,"4,65% a.a.","80,1 hab/ha","5,30 SM/mês",NaN
92,Vila João Pessoa,7 dez 1959,84 ha,10522.0,"-0,23% a.a.","125,3 hab/ha","5,58 SM/mês",NaN
93,Vila Nova,7 dez 1959,1031 ha,33145.0,"1,46% a.a.","32,1 hab/ha","5,35 SM/mês",NaN
94,Vila São José,7 dez 1959,212 ha,28957.0,"0,93% a.a.","136,6 hab/ha","3,60 SM/mês",NaN


###1.2.2 Ativar se não tiver o arquivo HTML

In [ ]:
# url_wiki = "https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Porto_Alegre"
# response = requests.get(url_wiki)
# soup = BeautifulSoup(response.content,'html.parser')
# html_tables = soup.find_all('table','wikitable sortable')
# for row in html_tables[0].find_all('tr')[:-1]:
#     if row.th:
#         column_names = [i.text.rstrip('\n').strip()
#                         for i in row.find_all('th')]
#         df = pd.DataFrame(columns=column_names)
#     else:
#         dict_row = {column_names[k]:v.text.rstrip('\n').strip() 
#                     for k,v in enumerate(row.find_all('td'))}
#         df = df.append(dict_row, ignore_index=True)
# df = df.replace('',np.nan)
# df

###1.2.3 Seleção de informações relevantes e tratamento de nulos

Isso precisa ser feito antes da limpeza dos dados porque é necessário definir a lista de bairros para encontrar as coordenadas e passar para a API do Foursquare.

In [16]:
df.drop(['Nota','Data de Criação','Tx Cresc Pop 91-00'],axis=1,inplace=True)
df.rename(columns={
    'Bairro':'bairro',
    'Área':'area',
    'População2010':'pop2010',
    'Densidade':'densidade',
    'Renda média por domicílio':'renda'
},inplace=True)

In [17]:
df

,bairro,area,pop2010,densidade,renda
0,Aberta dos Morros,NaN,NaN,NaN,NaN
1,Agronomia,1241 ha,12222.0,"8,6 hab/ha","3,98 SM/mês"
2,Anchieta,84 ha,203.0,"2,4 hab/ha","8,41 SM/mês"
3,Arquipélago,4718 ha,5061.0,"1,1 hab/ha","2,96 SM/mês"
4,Auxiliadora,82 ha,9985.0,"121,8 hab/ha","19,57 SM/mês"
...,...,...,...,...,...
91,Vila Jardim,178 ha,14251.0,"80,1 hab/ha","5,30 SM/mês"
92,Vila João Pessoa,84 ha,10522.0,"125,3 hab/ha","5,58 SM/mês"
93,Vila Nova,1031 ha,33145.0,"32,1 hab/ha","5,35 SM/mês"
94,Vila São José,212 ha,28957.0,"136,6 hab/ha","3,60 SM/mês"


In [18]:
df.isnull().sum()

bairro        0
area         15
pop2010      15
densidade    16
renda        16
dtype: int64

In [19]:
linhas_com_nulos = df[df.isnull().any(1)]
display(linhas_com_nulos.shape, linhas_com_nulos)

(17, 5)

,bairro,area,pop2010,densidade,renda
0,Aberta dos Morros,NaN,NaN,NaN,NaN
10,Boa Vista do Sul,2436 ha,2309.0,NaN,"2,28 SM/mês"
18,Chapéu do Sol,NaN,NaN,NaN,NaN
22,Costa e Silva,NaN,NaN,NaN,NaN
26,Extrema,NaN,NaN,NaN,NaN
39,Jardim Dona Leopoldina,NaN,NaN,NaN,NaN
41,Jardim Isabel,NaN,NaN,NaN,NaN
42,Jardim Itu,NaN,NaN,NaN,NaN
45,Jardim Sabará,NaN,NaN,NaN,NaN
56,Morro Santana,NaN,NaN,NaN,NaN


Pode-se notar que:
* Todas as linhas que possuem algum valor nulo estão no dataframe `linhas_com_nulos`;
* A densidade populacional de Boa Vista do Sul pode ser calculada posteriormente dividindo-se `pop2010` por `area`;
* A linha com bairro `Zona Indefinida` pode ser eliminada.

In [20]:
df = df[df['bairro']!='Zona Indefinida'].reset_index().drop('index',axis=1)

Excetuando-se a linha do bairro Boa Vista do Sul, todas as outras com valores nulos serão guardadas em um dataframe separado chamado `bairros_sem_info`.

In [21]:
bairros_sem_info = linhas_com_nulos[
    (linhas_com_nulos['bairro'] != 'Zona Indefinida') &
    (linhas_com_nulos['bairro'] != 'Boa Vista do Sul')
].reset_index().drop('index',axis=1)
display(bairros_sem_info.shape, bairros_sem_info)

(15, 5)

,bairro,area,pop2010,densidade,renda
0,Aberta dos Morros,NaN,NaN,NaN,NaN
1,Chapéu do Sol,NaN,NaN,NaN,NaN
2,Costa e Silva,NaN,NaN,NaN,NaN
3,Extrema,NaN,NaN,NaN,NaN
4,Jardim Dona Leopoldina,NaN,NaN,NaN,NaN
5,Jardim Isabel,NaN,NaN,NaN,NaN
6,Jardim Itu,NaN,NaN,NaN,NaN
7,Jardim Sabará,NaN,NaN,NaN,NaN
8,Morro Santana,NaN,NaN,NaN,NaN
9,Parque Santa Fé,NaN,NaN,NaN,NaN


In [22]:
display(df.isnull().sum(), df)

bairro        0
area         15
pop2010      15
densidade    16
renda        15
dtype: int64

,bairro,area,pop2010,densidade,renda
0,Aberta dos Morros,NaN,NaN,NaN,NaN
1,Agronomia,1241 ha,12222.0,"8,6 hab/ha","3,98 SM/mês"
2,Anchieta,84 ha,203.0,"2,4 hab/ha","8,41 SM/mês"
3,Arquipélago,4718 ha,5061.0,"1,1 hab/ha","2,96 SM/mês"
4,Auxiliadora,82 ha,9985.0,"121,8 hab/ha","19,57 SM/mês"
...,...,...,...,...,...
90,Vila Ipiranga,520 ha,20951.0,"40,3 hab/ha","8,99 SM/mês"
91,Vila Jardim,178 ha,14251.0,"80,1 hab/ha","5,30 SM/mês"
92,Vila João Pessoa,84 ha,10522.0,"125,3 hab/ha","5,58 SM/mês"
93,Vila Nova,1031 ha,33145.0,"32,1 hab/ha","5,35 SM/mês"


Além disso, o bairro Centro deve se chamar Centro Histórico para facilitar a busca pelas coordenadas:

In [23]:
df['bairro'][df['bairro']=='Centro'] = 'Centro Histórico'
df[
   (df['bairro']=='Centro Histórico') |
   (df['bairro']=='Centro')
]

,bairro,area,pop2010,densidade,renda
17,Centro Histórico,228 ha,36862.0,"161,7 hab/ha","12,61 SM/mês"


##1.3 Obtenção das coordenadas

Lista de bairros a serem considerados:

In [24]:
lista_bairros = df['bairro'].map(lambda x: x.strip()).tolist()
# lista_bairros

###1.3.1 Limpeza preliminar para obter as áreas dos bairros em metros quadrados

A transformação obedece à seguinte relação:

$x [m^2]=10^4x [ha]$

In [25]:
area_m2 = lambda x: int(x.rstrip(' ha'))*(10**4) if (type(x)==str and x[-2:]=='ha') else x
df['area'] = df['area'].map(area_m2)
df

,bairro,area,pop2010,densidade,renda
0,Aberta dos Morros,NaN,NaN,NaN,NaN
1,Agronomia,12410000.0,12222.0,"8,6 hab/ha","3,98 SM/mês"
2,Anchieta,840000.0,203.0,"2,4 hab/ha","8,41 SM/mês"
3,Arquipélago,47180000.0,5061.0,"1,1 hab/ha","2,96 SM/mês"
4,Auxiliadora,820000.0,9985.0,"121,8 hab/ha","19,57 SM/mês"
...,...,...,...,...,...
90,Vila Ipiranga,5200000.0,20951.0,"40,3 hab/ha","8,99 SM/mês"
91,Vila Jardim,1780000.0,14251.0,"80,1 hab/ha","5,30 SM/mês"
92,Vila João Pessoa,840000.0,10522.0,"125,3 hab/ha","5,58 SM/mês"
93,Vila Nova,10310000.0,33145.0,"32,1 hab/ha","5,35 SM/mês"


Com isso pode-se calcular os raios dos círculos que possuem a mesma área de cada bairro, a fim de representá-los:

$A=\pi r^2\quad⟹\quad r=\sqrt{\dfrac{A}{\pi}\;}$

In [26]:
df['raio'] = df['area'].map(lambda x: np.sqrt(x/np.pi))
raio_medio = df['raio'].mean()
display(f'Raio médio: {raio_medio} metros', df)

'Raio médio: 1040.1301591429885 metros'

,bairro,area,pop2010,densidade,renda,raio
0,Aberta dos Morros,NaN,NaN,NaN,NaN,NaN
1,Agronomia,12410000.0,12222.0,"8,6 hab/ha","3,98 SM/mês",1987.517468
2,Anchieta,840000.0,203.0,"2,4 hab/ha","8,41 SM/mês",517.088295
3,Arquipélago,47180000.0,5061.0,"1,1 hab/ha","2,96 SM/mês",3875.288432
4,Auxiliadora,820000.0,9985.0,"121,8 hab/ha","19,57 SM/mês",510.895397
...,...,...,...,...,...,...
90,Vila Ipiranga,5200000.0,20951.0,"40,3 hab/ha","8,99 SM/mês",1286.550197
91,Vila Jardim,1780000.0,14251.0,"80,1 hab/ha","5,30 SM/mês",752.722789
92,Vila João Pessoa,840000.0,10522.0,"125,3 hab/ha","5,58 SM/mês",517.088295
93,Vila Nova,10310000.0,33145.0,"32,1 hab/ha","5,35 SM/mês",1811.566981


###1.3.2 Uilizando os geocoders

In [27]:
def latlon(bairro):
    coord = None
    while(coord is None):
        g = geocoder.arcgis(f'{bairro}, Porto Alegre, Brasil')
        coord = g.latlng
    return coord

Mesclando geocoder e geopy: 

(Ativar se não possuir o CSV.)

In [ ]:
# geolocator = Nominatim(user_agent='TestBot')
# latitudes = []
# longitudes = []
# errados = [
#     'Camaquã',
#     'Cristal',
#     'Menino Deus',
#     'Morro Santana',
#     'Santo Antônio',
#     'Vila Ipiranga'
# ]
# for bairro in lista_bairros:
#     if bairro in errados:
#         g = latlon(bairro)
#         latitudes.append(g[0])
#         longitudes.append(g[1])
#     else:
#         g = geolocator.geocode(f"{bairro}, Porto Alegre, Brasil")
#         latitudes.append(g.latitude)
#         longitudes.append(g.longitude)
# df['lat'] = latitudes
# df['lon'] = longitudes
# # df.to_csv(caminho + 'bairros_geoloc_3.csv')

Resetando o dataframe com base no CSV gerado para não depender da requisição do geopy:

In [29]:
df = pd.read_csv(caminho + 'bairros_geoloc_3.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,bairro,area,pop2010,densidade,renda,raio,lat,lon
0,Aberta dos Morros,NaN,NaN,NaN,NaN,NaN,-30.160022,-51.197486
1,Agronomia,12410000.0,12222.0,"8,6 hab/ha","3,98 SM/mês",1987.517468,-30.069267,-51.149217
2,Anchieta,840000.0,203.0,"2,4 hab/ha","8,41 SM/mês",517.088295,-29.975652,-51.174903
3,Arquipélago,47180000.0,5061.0,"1,1 hab/ha","2,96 SM/mês",3875.288432,-29.992760,-51.226618
4,Auxiliadora,820000.0,9985.0,"121,8 hab/ha","19,57 SM/mês",510.895397,-30.020011,-51.190588
...,...,...,...,...,...,...,...,...
90,Vila Ipiranga,5200000.0,20951.0,"40,3 hab/ha","8,99 SM/mês",1286.550197,-30.022920,-51.153680
91,Vila Jardim,1780000.0,14251.0,"80,1 hab/ha","5,30 SM/mês",752.722789,-30.035177,-51.153283
92,Vila João Pessoa,840000.0,10522.0,"125,3 hab/ha","5,58 SM/mês",517.088295,-30.065375,-51.176165
93,Vila Nova,10310000.0,33145.0,"32,1 hab/ha","5,35 SM/mês",1811.566981,-30.116003,-51.207517


In [30]:
df.isnull().sum() # Todas as coordenadas foram obtidas com sucesso.

bairro        0
area         15
pop2010      15
densidade    16
renda        15
raio         15
lat           0
lon           0
dtype: int64

###1.3.3 Visualizando com `folium` para verificar as coordenadas

Definindo função que retorna o mapa com os círculos e marcações:

In [31]:
def add_circ_mark(my_map, my_data):
    for k,row in my_data.iterrows():
        coord = (row['lat'], row['lon'])
        bairro = row['bairro']
        raio = row['raio_fill']
        marker = folium.Marker(coord, popup=bairro)
        circle = folium.Circle(coord, radius=raio, color='#d35400', fill=True)
        my_map.add_child(marker).add_child(circle)
    return my_map

Obtendo e salvando coordenadas de Porto Alegre para centralizar a visualização inicial do mapa:

In [32]:
# geoloc_poa = geolocator.geocode("Porto Alegre, Brazil")
# coord_poa = (geoloc_poa.latitude, geoloc_poa.longitude)
# # Salvando...
coord_poa = (-30.0324999, -51.2303767) 
print(coord_poa)

(-30.0324999, -51.2303767)


Ajustes manuais para alguns raios:

In [33]:
raio_std = df['raio'].std()

# Raio do bairro Arquipélago estava muito grande:
df['raio'][df['bairro']=='Arquipélago'] = raio_medio 

# Raio do bairro Cristal estava muito pequeno:
df['raio'][df['bairro']=='Cristal'] = raio_medio + raio_std/2 

# Aumentando todos em 50% para cobrir todo o território da cidade:
df['raio_fill'] = (df['raio'].fillna(raio_medio)*1.5).astype(int)

# df

In [34]:
enter_view = (coord_poa[0]-0.06, coord_poa[1]+0.05)
fig = folium.Figure(width=900, height=600)
map = folium.Map(location=enter_view, zoom_start=11).add_to(fig)
add_circ_mark(map, df)

##1.4 Utilizando API do Foursquare

Podemos identificar que, apesar dos problemas com as coordenadas ter sido resolvido com sucesso, a abordagem de utilizar um raio para cada bairro produz muita sobreposição e uma representação equivocada. Em vez disso, tentarei utilizar a própria API do Foursquare para obter informações de estabelecimentos da cidade toda, os quais serão agrupados com `groupby` pelos bairros.

my_id = " "

my_secret = " "

my_version = "20220430"

my_project_key_1 = " "

Função para salvar arquivos JSON:

In [36]:
def salva_json(content, nome):
    # # Ativar se não tiver um caminho global:
    # caminho = '/content/drive/MyDrive/Projeto_Foursquare/'
    arq_json = open(caminho + f'{nome}.json', 'w') 
    json.dump(content.json(), arq_json, indent = 4)     
    arq_json.close()
    fuso_horario = timezone(timedelta(hours=-3))
    agora = datetime.now().astimezone(fuso_horario)
    print(f'Salvo em: {agora.strftime("%d/%m/%Y %H:%M")}')

# arquivo_json = open(caminho + 'categorias_resposta.json', 'w') 
# json.dump(resposta_cat, arquivo_json, indent = 4)     
# arquivo_json.close() 

###1.4.1 Obtendo os códigos das categorias

Foi descontinuado. Agora as categorias maiores dependem só dos dois primeiros dígitos do código das categorias menores. 

Requisitando a API:

In [ ]:
# url_cat_base = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'
# url_cat = url_cat_base.format(my_id, my_secret, my_version)     
# resposta_cat = requests.get(url_cat).json()
# resposta_cat = resposta_cat['response']['categories']

In [ ]:
# salva_json(resposta_cat, categorias_resposta)

Carregando o arquivo JSON das categorias para não depender da API:

In [37]:
arquivo_json = open(caminho + 'categorias_resposta.json','r')
resposta_cat = json.load(arquivo_json)
df_cat = pd.json_normalize(data=resposta_cat['response']['categories'])
arquivo_json.close()
display(df_cat.shape, df_cat)

(10, 7)

,id,name,pluralName,shortName,categories,icon.prefix,icon.suffix
0,4d4b7104d754a06370d81259,Arts & Entertainment,Arts & Entertainment,Arts & Entertainment,"[{'id': '56aa371be4b08b9a8d5734db', 'name': 'A...",https://ss3.4sqi.net/img/categories_v2/arts_en...,.png
1,4d4b7105d754a06372d81259,College & University,Colleges & Universities,College & Education,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",https://ss3.4sqi.net/img/categories_v2/educati...,.png
2,4d4b7105d754a06373d81259,Event,Events,Event,"[{'id': '52f2ab2ebcbc57f1066b8b3b', 'name': 'C...",https://ss3.4sqi.net/img/categories_v2/event/d...,.png
3,4d4b7105d754a06374d81259,Food,Food,Food,"[{'id': '503288ae91d4c4b30a586d67', 'name': 'A...",https://ss3.4sqi.net/img/categories_v2/food/de...,.png
4,4d4b7105d754a06376d81259,Nightlife Spot,Nightlife Spots,Nightlife,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",https://ss3.4sqi.net/img/categories_v2/nightli...,.png
5,4d4b7105d754a06377d81259,Outdoors & Recreation,Outdoors & Recreation,Outdoors & Recreation,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",https://ss3.4sqi.net/img/categories_v2/parks_o...,.png
6,4d4b7105d754a06375d81259,Professional & Other Places,Professional & Other Places,Professional,"[{'id': '4e52d2d203646f7c19daa8ae', 'name': 'A...",https://ss3.4sqi.net/img/categories_v2/buildin...,.png
7,4e67e38e036454776db1fb3a,Residence,Residences,Residence,"[{'id': '5032891291d4c4b30a586d68', 'name': 'A...",https://ss3.4sqi.net/img/categories_v2/buildin...,.png
8,4d4b7105d754a06378d81259,Shop & Service,Shops & Services,Shops,"[{'id': '52f2ab2ebcbc57f1066b8b56', 'name': 'A...",https://ss3.4sqi.net/img/categories_v2/shops/d...,.png
9,4d4b7105d754a06379d81259,Travel & Transport,Travel & Transport,Travel,"[{'id': '4bf58dd8d48988d1ed931735', 'name': 'A...",https://ss3.4sqi.net/img/categories_v2/travel/...,.png


Filtrando apenas o necessário e renomeando colunas:

In [38]:
df_cat = df_cat[['id','pluralName']]
df_cat = df_cat.rename(columns={'id':'categoria_id','pluralName':'categoria'})
# df_cat.to_csv(caminho + 'categorias_dataframe.csv')
df_cat

,categoria_id,categoria
0,4d4b7104d754a06370d81259,Arts & Entertainment
1,4d4b7105d754a06372d81259,Colleges & Universities
2,4d4b7105d754a06373d81259,Events
3,4d4b7105d754a06374d81259,Food
4,4d4b7105d754a06376d81259,Nightlife Spots
5,4d4b7105d754a06377d81259,Outdoors & Recreation
6,4d4b7105d754a06375d81259,Professional & Other Places
7,4e67e38e036454776db1fb3a,Residences
8,4d4b7105d754a06378d81259,Shops & Services
9,4d4b7105d754a06379d81259,Travel & Transport


In [39]:
lista_cat_sup = df_cat['categoria'].tolist()
lista_cat_sup

['Arts & Entertainment',
 'Colleges & Universities',
 'Events',
 'Food',
 'Nightlife Spots',
 'Outdoors & Recreation',
 'Professional & Other Places',
 'Residences',
 'Shops & Services',
 'Travel & Transport']

In [40]:
dict_cat = dict(zip(range(10,20),lista_cat_sup))
dict_cat

{10: 'Arts & Entertainment',
 11: 'Colleges & Universities',
 12: 'Events',
 13: 'Food',
 14: 'Nightlife Spots',
 15: 'Outdoors & Recreation',
 16: 'Professional & Other Places',
 17: 'Residences',
 18: 'Shops & Services',
 19: 'Travel & Transport'}

###1.4.2 Obtendo informações sobre locais de cada categoria para cada bairro

Modo antigo (v2):

In [ ]:
# lat, lon = -30.117536111111, -51.1219
# raio = 18000
# url_base = 'https://api.foursquare.com/v2/venues/search?'
# url_join = '&'.join(['client_id={}', 
#                      'client_secret={}', 
#                      'v={}',
#                      'll={},{}', 
#                      'radius={}'])
# url_explore = (url_base)+(url_join).format(my_id,
#                                            my_secret,
#                                            my_version,
#                                            lat, lon, 
#                                            raio)
# # print(url_explore)
# headers = {"Accept": "application/json"}


# # response = requests.get(url, headers=headers)
# try:
#     response = requests.get(url_explore, headers=headers).json()#['response']['totalResults']
# except:
#     response = 0
# response

{'meta': {'code': 410,
  'errorDetail': 'Usage of the V2 Places API has been deprecated for new Projects. Please see our updated documentation for V3 for more details: https://docs.foursquare.com/reference',
  'errorType': 'deprecated',
  'requestId': '626df6d173655f250f08aca8'},
 'response': {}}

Modo novo (v3):

In [ ]:
# # Ativar apenas se for fazer nova solicitação de teste
# lat,lon = -30.117536111111,-51.1219
# raio = 18000
# url_base = 'https://api.foursquare.com/v3/places/search'
# url_search = '?ll={}%2C{}&radius={}&limit{}' # &categories={}
# url = url_base + url_search.format(lat,lon,raio,50)
# headers = {
#     "Accept": "application/json",
#     "Authorization": my_project_key_1
# }
# response = requests.request("GET", url, headers=headers)
# df_teste = pd.json_normalize(data=response.json()['results'])
# df_teste

Salvando:

In [ ]:
# arquivo_json = open(caminho + 'testando_resposta.json', 'w') 
# json.dump(response.json(), arquivo_json, indent = 4)     
# arquivo_json.close() 

Resetando para não depender do request:

In [41]:
arquivo_json = open(caminho + 'testando_resposta.json','r')
resposta_teste = json.load(arquivo_json)
df_teste = pd.json_normalize(data=resposta_teste['results'])
arquivo_json.close() 
df_teste

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,related_places.parent.fsq_id,related_places.parent.name,geocodes.roof.latitude,geocodes.roof.longitude,location.post_town,location.address_extended
0,50147cbde4b06b8dcc50aa61,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",[],4599,/v3/places/50147cbde4b06b8dcc50aa61,Xis da Rosane,America/Sao_Paulo,-30.094535,-51.082121,"Rua Ns de Fatima, 55 Viamopolis",BR,,"Rua Ns de Fatima, 55 Viamopolis, Viamão, RS, 9...",Viamão,[Viamópolis],94470-500,RS,NaN,NaN,NaN,NaN,NaN,NaN
1,530e0cf4498e140e062570cb,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],5003,/v3/places/530e0cf4498e140e062570cb,Café da Física,America/Sao_Paulo,-30.072553,-51.119723,UFRGS - Campus do Vale,BR,Inst. de Física,"UFRGS - Campus do Vale (Inst. de Física), Port...",Porto Alegre,NaN,90540-000,RS,4ce6fa32f86537049a4ca6c4,Instituto de Física,NaN,NaN,NaN,NaN
2,4d51df4cd7eaa143290b780f,"[{'id': 16027, 'name': 'Mountain', 'icon': {'p...",[],4694,/v3/places/4d51df4cd7eaa143290b780f,Morro Ricaldone,America/Sao_Paulo,-30.158329,-51.134588,Rua Santo Inácio,BR,Rua Engenheiro Alvaro Nunes Pereira,Rua Santo Inácio (Rua Engenheiro Alvaro Nunes ...,Porto Alegre,NaN,90570-150,RS,NaN,NaN,-30.158329,-51.134588,NaN,NaN
3,4ebfd1299adfe2b7b6292fb0,"[{'id': 17067, 'name': 'Fruit and Vegetable St...",[],6002,/v3/places/4ebfd1299adfe2b7b6292fb0,Sacolão da Chácara,America/Sao_Paulo,-30.160339,-51.159827,"Avenida Edgar Pires de Castro, 3295",BR,,"Avenida Edgar Pires de Castro, 3295, Porto Ale...",Porto Alegre,[Hípica],91787-000,RS,NaN,NaN,-30.160339,-51.159827,Aberta Morros,NaN
4,52c9dbcb498ecccd1632e3e0,"[{'id': 10000, 'name': 'Arts and Entertainment...",[],5412,/v3/places/52c9dbcb498ecccd1632e3e0,Terrenão,America/Sao_Paulo,-30.084500,-51.080535,NaN,BR,,Rio Grande do Sul,Viamópolis,NaN,,RS,NaN,NaN,NaN,NaN,NaN,NaN
5,4fdbddede4b09473e13877c6,"[{'id': 13058, 'name': 'Hot Dog Joint', 'icon'...",[],5146,/v3/places/4fdbddede4b09473e13877c6,Cachorro Americano,America/Sao_Paulo,-30.072851,-51.108332,Avenida Liberdade,BR,,"Avenida Liberdade, Porto Alegre, RS",Porto Alegre,NaN,,RS,NaN,NaN,NaN,NaN,NaN,NaN
6,4fbf9f5fe4b0c9fe0515fe44,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",[{'id': 'ab4bee40-d68a-012e-5619-003048cad9da'...,6185,/v3/places/4fbf9f5fe4b0c9fe0515fe44,Subway,America/Sao_Paulo,-30.165355,-51.154999,"Avenida Edgar Pires de Castro, 4100",BR,NaN,"Avenida Edgar Pires de Castro, 4100, Porto Ale...",Porto Alegre,NaN,91787-000,RS,NaN,NaN,-30.165355,-51.154999,NaN,lj 2
7,50797ae8e4b06fd3a9bcd1a4,"[{'id': 11061, 'name': 'Health and Beauty Serv...",[],4641,/v3/places/50797ae8e4b06fd3a9bcd1a4,Tutu Art Fashion,America/Sao_Paulo,-30.088034,-51.088071,Parada 40,BR,NaN,"Parada 40, Viamão, RS",Viamão,NaN,,RS,NaN,NaN,-30.088034,-51.088071,NaN,NaN
8,50bfd91ce4b0b9f403fae5f9,"[{'id': 13010, 'name': 'Dive Bar', 'icon': {'p...",[],5949,/v3/places/50bfd91ce4b0b9f403fae5f9,Boteco Cabritos,America/Sao_Paulo,-30.065356,-51.107594,"Avenida Liberdade, 1820",BR,,"Avenida Liberdade, 1820, Viamão, RS, 94480-500",Viamão,NaN,94480-500,RS,NaN,NaN,-30.065356,-51.107594,Santa Isabel,NaN
9,4e261d89c65bb6120fc90f75,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],5547,/v3/places/4e261d89c65bb6120fc90f75,Pôr do Sol Bier Bar,America/Sao_Paulo,-30.069222,-51.107067,"Rua Breno Cardia Alves, 58",BR,,"Rua Breno Cardia Alves, 58, Viamão, RS, 94480-640",Viamão,[Santa Isabel],94480-640,RS,NaN,NaN,NaN,NaN,Santa Isabel,NaN


Aprendendo a resgatar as categorias de cada lugar:

In [42]:
df_teste['categories'][0][0]['id'], df_teste['categories'][0][0]['name']

(13031, 'Burger Joint')

In [43]:
for i in df_teste['categories']:
    print(i[0]['id'],':',i[0]['name'])

13031 : Burger Joint
13034 : Café
16027 : Mountain
17067 : Fruit and Vegetable Store
10000 : Arts and Entertainment
13058 : Hot Dog Joint
13145 : Fast Food Restaurant
11061 : Health and Beauty Service
13010 : Dive Bar
13003 : Bar


In [44]:
teste1 = []
teste2 = []
for row in df_teste['categories']:
    teste1.append(row[0]['name'])
    teste2.append(row[0]['id'])
df_teste['cat_nome'] = teste1
df_teste['cat_id'] = teste2
dig2 = lambda x: int(str(x)[:2])
df_teste['cat_sup'] = df_teste['cat_id'].map(dig2).map(dict_cat)
df_teste

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,related_places.parent.fsq_id,related_places.parent.name,geocodes.roof.latitude,geocodes.roof.longitude,location.post_town,location.address_extended,cat_nome,cat_id,cat_sup
0,50147cbde4b06b8dcc50aa61,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",[],4599,/v3/places/50147cbde4b06b8dcc50aa61,Xis da Rosane,America/Sao_Paulo,-30.094535,-51.082121,"Rua Ns de Fatima, 55 Viamopolis",BR,,"Rua Ns de Fatima, 55 Viamopolis, Viamão, RS, 9...",Viamão,[Viamópolis],94470-500,RS,NaN,NaN,NaN,NaN,NaN,NaN,Burger Joint,13031,Food
1,530e0cf4498e140e062570cb,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],5003,/v3/places/530e0cf4498e140e062570cb,Café da Física,America/Sao_Paulo,-30.072553,-51.119723,UFRGS - Campus do Vale,BR,Inst. de Física,"UFRGS - Campus do Vale (Inst. de Física), Port...",Porto Alegre,NaN,90540-000,RS,4ce6fa32f86537049a4ca6c4,Instituto de Física,NaN,NaN,NaN,NaN,Café,13034,Food
2,4d51df4cd7eaa143290b780f,"[{'id': 16027, 'name': 'Mountain', 'icon': {'p...",[],4694,/v3/places/4d51df4cd7eaa143290b780f,Morro Ricaldone,America/Sao_Paulo,-30.158329,-51.134588,Rua Santo Inácio,BR,Rua Engenheiro Alvaro Nunes Pereira,Rua Santo Inácio (Rua Engenheiro Alvaro Nunes ...,Porto Alegre,NaN,90570-150,RS,NaN,NaN,-30.158329,-51.134588,NaN,NaN,Mountain,16027,Professional & Other Places
3,4ebfd1299adfe2b7b6292fb0,"[{'id': 17067, 'name': 'Fruit and Vegetable St...",[],6002,/v3/places/4ebfd1299adfe2b7b6292fb0,Sacolão da Chácara,America/Sao_Paulo,-30.160339,-51.159827,"Avenida Edgar Pires de Castro, 3295",BR,,"Avenida Edgar Pires de Castro, 3295, Porto Ale...",Porto Alegre,[Hípica],91787-000,RS,NaN,NaN,-30.160339,-51.159827,Aberta Morros,NaN,Fruit and Vegetable Store,17067,Residences
4,52c9dbcb498ecccd1632e3e0,"[{'id': 10000, 'name': 'Arts and Entertainment...",[],5412,/v3/places/52c9dbcb498ecccd1632e3e0,Terrenão,America/Sao_Paulo,-30.084500,-51.080535,NaN,BR,,Rio Grande do Sul,Viamópolis,NaN,,RS,NaN,NaN,NaN,NaN,NaN,NaN,Arts and Entertainment,10000,Arts & Entertainment
5,4fdbddede4b09473e13877c6,"[{'id': 13058, 'name': 'Hot Dog Joint', 'icon'...",[],5146,/v3/places/4fdbddede4b09473e13877c6,Cachorro Americano,America/Sao_Paulo,-30.072851,-51.108332,Avenida Liberdade,BR,,"Avenida Liberdade, Porto Alegre, RS",Porto Alegre,NaN,,RS,NaN,NaN,NaN,NaN,NaN,NaN,Hot Dog Joint,13058,Food
6,4fbf9f5fe4b0c9fe0515fe44,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",[{'id': 'ab4bee40-d68a-012e-5619-003048cad9da'...,6185,/v3/places/4fbf9f5fe4b0c9fe0515fe44,Subway,America/Sao_Paulo,-30.165355,-51.154999,"Avenida Edgar Pires de Castro, 4100",BR,NaN,"Avenida Edgar Pires de Castro, 4100, Porto Ale...",Porto Alegre,NaN,91787-000,RS,NaN,NaN,-30.165355,-51.154999,NaN,lj 2,Fast Food Restaurant,13145,Food
7,50797ae8e4b06fd3a9bcd1a4,"[{'id': 11061, 'name': 'Health and Beauty Serv...",[],4641,/v3/places/50797ae8e4b06fd3a9bcd1a4,Tutu Art Fashion,America/Sao_Paulo,-30.088034,-51.088071,Parada 40,BR,NaN,"Parada 40, Viamão, RS",Viamão,NaN,,RS,NaN,NaN,-30.088034,-51.088071,NaN,NaN,Health and Beauty Service,11061,Colleges & Universities
8,50bfd91ce4b0b9f403fae5f9,"[{'id': 13010, 'name': 'Dive Bar', 'icon': {'p...",[],5949,/v3/places/50bfd91ce4b0b9f403fae5f9,Boteco Cabritos,America/Sao_Paulo,-30.065356,-51.107594,"Avenida Liberdade, 1820",BR,,"Avenida Liberdade, 1820, Viamão, RS, 94480-500",Viamão,NaN,94480-500,RS,NaN,NaN,-30.065356,-51.107594,Santa Isabel,NaN,Dive Bar,13010,Food
9,4e261d89c65bb6120fc90f75,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],5547,/v3/places/4e261d89c65bb6120fc90f75,Pôr do Sol Bier Bar,America/Sao_Paulo,-30.069222,-51.107067,"Rua Breno Cardia Alves, 58",BR,,"Rua Breno Cardia Alves, 58, Viamão, RS, 94480-640",Viamão,[Santa Isabel],94480-640,RS,NaN,NaN,NaN,NaN,Santa Isabel,NaN,Bar,13003,Fo

In [45]:
manter_colunas = [
    'fsq_id',
    'categories',
    'name',
    'geocodes.main.latitude',
    'geocodes.main.longitude',
    'location.formatted_address',
    'location.locality',
    'location.neighborhood'
]
dict_cols = {
    'categories':'categorias',
    'name':'nome',
    'geocodes.main.latitude':'lat',
    'geocodes.main.longitude':'lon',
    'location.formatted_address':'endereco',
    'location.locality':'cidade',
    'location.neighborhood':'bairro'
}
testando = df_teste[manter_colunas].rename(columns=dict_cols)
testando
# testando[list(set(testando.columns.tolist())-set(['categorias']))]

,fsq_id,categorias,nome,lat,lon,endereco,cidade,bairro
0,50147cbde4b06b8dcc50aa61,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",Xis da Rosane,-30.094535,-51.082121,"Rua Ns de Fatima, 55 Viamopolis, Viamão, RS, 9...",Viamão,[Viamópolis]
1,530e0cf4498e140e062570cb,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Café da Física,-30.072553,-51.119723,"UFRGS - Campus do Vale (Inst. de Física), Port...",Porto Alegre,NaN
2,4d51df4cd7eaa143290b780f,"[{'id': 16027, 'name': 'Mountain', 'icon': {'p...",Morro Ricaldone,-30.158329,-51.134588,Rua Santo Inácio (Rua Engenheiro Alvaro Nunes ...,Porto Alegre,NaN
3,4ebfd1299adfe2b7b6292fb0,"[{'id': 17067, 'name': 'Fruit and Vegetable St...",Sacolão da Chácara,-30.160339,-51.159827,"Avenida Edgar Pires de Castro, 3295, Porto Ale...",Porto Alegre,[Hípica]
4,52c9dbcb498ecccd1632e3e0,"[{'id': 10000, 'name': 'Arts and Entertainment...",Terrenão,-30.084500,-51.080535,Rio Grande do Sul,Viamópolis,NaN
5,4fdbddede4b09473e13877c6,"[{'id': 13058, 'name': 'Hot Dog Joint', 'icon'...",Cachorro Americano,-30.072851,-51.108332,"Avenida Liberdade, Porto Alegre, RS",Porto Alegre,NaN
6,4fbf9f5fe4b0c9fe0515fe44,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",Subway,-30.165355,-51.154999,"Avenida Edgar Pires de Castro, 4100, Porto Ale...",Porto Alegre,NaN
7,50797ae8e4b06fd3a9bcd1a4,"[{'id': 11061, 'name': 'Health and Beauty Serv...",Tutu Art Fashion,-30.088034,-51.088071,"Parada 40, Viamão, RS",Viamão,NaN
8,50bfd91ce4b0b9f403fae5f9,"[{'id': 13010, 'name': 'Dive Bar', 'icon': {'p...",Boteco Cabritos,-30.065356,-51.107594,"Avenida Liberdade, 1820, Viamão, RS, 94480-500",Viamão,NaN
9,4e261d89c65bb6120fc90f75,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",Pôr do Sol Bier Bar,-30.069222,-51.107067,"Rua Breno Cardia Alves, 58, Viamão, RS, 94480-640",Viamão,[Santa Isabel]


Testando `reverse` do `geopy`:

In [46]:
geolocator = Nominatim(user_agent='TestBot')
my_address = geolocator.reverse((-30.033183333333, -51.234405555556), addressdetails=True)
my_address.raw

{'address': {'city': 'Porto Alegre',
  'city_district': 'Porto Alegre',
  'country': 'Brasil',
  'country_code': 'br',
  'county': 'Região Metropolitana de Porto Alegre',
  'municipality': 'Região Geográfica Imediata de Porto Alegre',
  'postcode': '90010-282',
  'region': 'Região Sul',
  'road': 'Rua General Bento Martins',
  'shop': 'Mercado Moinho',
  'state': 'Rio Grande do Sul',
  'state_district': 'Região Geográfica Intermediária de Porto Alegre',
  'suburb': 'Centro Histórico'},
 'boundingbox': ['-30.0335217', '-30.0334217', '-51.2343339', '-51.2342339'],
 'display_name': 'Mercado Moinho, Rua General Bento Martins, Centro Histórico, Porto Alegre, Região Geográfica Imediata de Porto Alegre, Região Metropolitana de Porto Alegre, Região Geográfica Intermediária de Porto Alegre, Rio Grande do Sul, Região Sul, 90010-282, Brasil',
 'lat': '-30.0334717',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '-51.2342839',
 'osm_id': 2387825587,
 

In [47]:
my_address.raw['address']['suburb']

'Centro Histórico'

Definindo funções para várias requisições:

In [48]:
tira_lista = lambda x: x[0] if (pd.isnull(x)==False and type(x)==list) else x

def obter_amostra(lat, lon, raio, limit):
    url_base = "https://api.foursquare.com/v3/places/search"
    url_search = "?ll={}%2C{}&radius={}&limit{}" # &categories={}
    heads = {"Accept": "application/json", "Authorization": my_project_key_1}
    url = url_base + url_search.format(lat, lon, raio, limit)
    response = requests.request("GET", url, headers=heads)
    # time.sleep(0.1)
    df_amostra = pd.json_normalize(response.json()['results'])
    # time.sleep(0.1)
    return df_amostra

def todas_amostras(dataframe, n_por_centroide, salva=True):
    df_todas = pd.DataFrame()
    N = df_todas.shape[0]
    # while N < n_por_centroide*(dataframe.shape[0]):
    for k,row in dataframe.iterrows():
        lat, lon, raio = row['lat'], row['lon'], row['raio_fill']
        # time.sleep(0.1)
        amostra = obter_amostra(lat, lon, raio, 50)
        # time.sleep(0.1)
        df_todas = pd.concat([df_todas, amostra], axis=0)
    manter_colunas = [
        'fsq_id',
        'categories',
        'name',
        'geocodes.main.latitude',
        'geocodes.main.longitude',
        'location.formatted_address',
        'location.locality',
        'location.neighborhood'
    ]
    dict_cols = {
        'categories':'categorias',
        'name':'nome',
        'geocodes.main.latitude':'lat',
        'geocodes.main.longitude':'lon',
        'location.formatted_address':'endereco',
        'location.locality':'cidade',
        'location.neighborhood':'bairro'
    }
    df_todas = df_todas[manter_colunas].rename(columns=dict_cols)
    df_todas['bairro'] = df_todas['bairro'].map(tira_lista)
    df_todas['fsq_id'] = df_todas['fsq_id'].map(tira_lista)
    df_todas.drop_duplicates(subset=['fsq_id'], inplace=True, ignore_index=True)
    cond_poa = df_todas['cidade']=='Porto Alegre'
    cond_nan = df_todas['cidade'].isnull()
    df_todas = df_todas[(cond_poa)|(cond_nan)]
    df_todas = df_todas.reset_index().drop('index', axis=1)
    N = df_todas.shape[0]
    if salva == True:
        # # Ativar se não tiver um caminho global:
        # caminho = '/content/drive/MyDrive/Projeto_Foursquare/'
        df_todas.to_csv(caminho + 'todas_amostras.csv')
        print('Salvo')
    return df_todas

def tratando(df_todas, salva=True):
    lista_cat_nome = []
    lista_cat_id = []
    for row in df_todas['categorias']:
        if row == []:
            lista_cat_nome.append(np.nan)
            lista_cat_id.append(np.nan)
        else:
            lista_cat_nome.append(row[0]['name'])
            lista_cat_id.append(row[0]['id'])
    df_todas['cat_nome'] = lista_cat_nome
    df_todas['cat_id'] = lista_cat_id
    dig2 = lambda x: int(str(x)[:2])
    dict_cat = dict(zip(range(10,20),lista_cat_sup))
    df_todas['cat_sup'] = df_todas['cat_id'].map(dig2,na_action='ignore')
    df_todas['cat_sup'] = df_todas['cat_sup'].map(dict_cat,na_action='ignore')    
    df_todas = df_todas[df_todas['cat_id'].isnull()==False]
    df_todas['cat_id'] = df_todas['cat_id'].astype(int)
    df_todas = df_todas.reset_index().drop('index',axis=1)
    if salva == True:
        # # Ativar se não tiver um caminho global:
        # caminho = '/content/drive/MyDrive/Projeto_Foursquare/'
        df_todas.to_csv(caminho + 'todas_amostras_tratadas.csv')
        print('Salvo')
    return df_todas

def obter_bairros(df_todas, salva=True):
    bairros = []
    geolocator = Nominatim(user_agent='TestBot')
    n_sem_bairro = 0
    sem_bairro = pd.DataFrame()
    for k,row in df_todas.iterrows():
        lat, lon = row['lat'], row['lon']
        address = geolocator.reverse((lat, lon), addressdetails=True)
        bairro = address.raw['address']['suburb']
        if bairro:
            bairros.append(bairro)
        elif row['bairro']:
            bairros.append(row['bairro'])
        else:
            bairros.append(np.nan)
            print(f'ATENÇÃO: linha {k} sem bairro!')
            n_sem_bairro = n_sem_bairro + 1
            sem_bairro = pd.concat([sem_bairro,row.to_frame()],axis=0)
    df_todas['bairro_geoloc'] = bairros
    if salva == True:
        # # Ativar se não tiver um caminho global:
        # caminho = '/content/drive/MyDrive/Projeto_Foursquare/'
        df_todas.to_csv(caminho + 'todas_amostras_tratadas_com_bairros.csv')
        print('Salvo')
    if n_sem_bairro > 0:
        print(f'\nTemos {n_sem_bairro} linhas sem bairro.\nListando:\n')
        display(sem_bairro)
    return df_todas

Fazendo as requisições:

(Ativar se não tiver o CSV.)

In [ ]:
# df_loc = todas_amostras(df, 10, salva=True)
# df_loc

Salvo


,fsq_id,categorias,nome,lat,lon,endereco,cidade,bairro
0,4c30afa6452620a139721f0f,"[{'id': 13083, 'name': 'Churrascaria', 'icon':...",Churrascaria Pastoriza,-30.158138,-51.194270,"Avenida Juca Batista, 4023, Porto Alegre, RS, ...",Porto Alegre,NaN
1,4d6a7586342b8cfad31cd72c,"[{'id': 13079, 'name': 'Brazilian Restaurant',...",Biskaia,-30.157387,-51.196786,"Avenida Juca Batista, 3782, Porto Alegre, RS, ...",Porto Alegre,NaN
2,4e7e51be61af0051f2f60efe,"[{'id': 17069, 'name': 'Grocery Store / Superm...",J. Carboni Supermercado,-30.157197,-51.197027,"Avenida Juca Batista, 3821, Porto Alegre, RS, ...",Porto Alegre,Hípica
3,590257a9e185f3577bacba57,"[{'id': 13048, 'name': 'Pie Shop', 'icon': {'p...",Bella Gula,-30.158703,-51.193424,"Zaffari, Porto Alegre, RS, 91770-001",Porto Alegre,Hípica
4,4f3a8595e4b01190396d76c1,"[{'id': 11045, 'name': 'Bank', 'icon': {'prefi...",CAIXA,-30.158300,-51.195022,"Rua Doutor Hermes Pacheco, 25 (Ag. Ag: 3481 - ...",Porto Alegre,NaN
...,...,...,...,...,...,...,...,...
841,595ce17df5e9d7161f191977,"[{'id': 17069, 'name': 'Grocery Store / Superm...",Mercado Muraro,-30.068700,-51.167293,"Rua Vinte e Seis de Dezembro, 245 (Vidal de ne...",Porto Alegre,NaN
842,4da87f41fa8c4175d0bb5320,"[{'id': 17110, 'name': 'Pet Supplies Store', '...",Aloha Pet Shop,-30.064177,-51.170898,"Rua Marechal Mallet, 401 (R. Saldanha da Gama)...",Porto Alegre,NaN
843,50ce3a60e4b03ff34722b153,"[{'id': 16032, 'name': 'Park', 'icon': {'prefi...",Parque do Iberê Camargo,-30.070915,-51.169867,,NaN,NaN
844,5849eedcd8f3e9271e4ecdbd,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",Burger King,-30.061245,-51.163762,"Carrefour, Porto Alegre, RS",Porto Alegre,NaN


Resetando `df_loc` para não depender das requisições:

In [49]:
df_loc = pd.read_csv(caminho + 'todas_amostras.csv', index_col='Unnamed: 0')
df_loc['categorias'] = df_loc['categorias'].map(lambda x: literal_eval(x))
df_loc

,fsq_id,categorias,nome,lat,lon,endereco,cidade,bairro
0,4c30afa6452620a139721f0f,"[{'id': 13083, 'name': 'Churrascaria', 'icon':...",Churrascaria Pastoriza,-30.158138,-51.194270,"Avenida Juca Batista, 4023, Porto Alegre, RS, ...",Porto Alegre,NaN
1,4d6a7586342b8cfad31cd72c,"[{'id': 13079, 'name': 'Brazilian Restaurant',...",Biskaia,-30.157387,-51.196786,"Avenida Juca Batista, 3782, Porto Alegre, RS, ...",Porto Alegre,NaN
2,4e7e51be61af0051f2f60efe,"[{'id': 17069, 'name': 'Grocery Store / Superm...",J. Carboni Supermercado,-30.157197,-51.197027,"Avenida Juca Batista, 3821, Porto Alegre, RS, ...",Porto Alegre,Hípica
3,590257a9e185f3577bacba57,"[{'id': 13048, 'name': 'Pie Shop', 'icon': {'p...",Bella Gula,-30.158703,-51.193424,"Zaffari, Porto Alegre, RS, 91770-001",Porto Alegre,Hípica
4,4f3a8595e4b01190396d76c1,"[{'id': 11045, 'name': 'Bank', 'icon': {'prefi...",CAIXA,-30.158300,-51.195022,"Rua Doutor Hermes Pacheco, 25 (Ag. Ag: 3481 - ...",Porto Alegre,NaN
...,...,...,...,...,...,...,...,...
841,595ce17df5e9d7161f191977,"[{'id': 17069, 'name': 'Grocery Store / Superm...",Mercado Muraro,-30.068700,-51.167293,"Rua Vinte e Seis de Dezembro, 245 (Vidal de ne...",Porto Alegre,NaN
842,4da87f41fa8c4175d0bb5320,"[{'id': 17110, 'name': 'Pet Supplies Store', '...",Aloha Pet Shop,-30.064177,-51.170898,"Rua Marechal Mallet, 401 (R. Saldanha da Gama)...",Porto Alegre,NaN
843,50ce3a60e4b03ff34722b153,"[{'id': 16032, 'name': 'Park', 'icon': {'prefi...",Parque do Iberê Camargo,-30.070915,-51.169867,NaN,NaN,NaN
844,5849eedcd8f3e9271e4ecdbd,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",Burger King,-30.061245,-51.163762,"Carrefour, Porto Alegre, RS",Porto Alegre,NaN


Linhas sem dados de categorias:

In [50]:
for k,row in enumerate(df_loc['categorias']):
    if row == []:
        print(row, f'linha {k}')

[] linha 209
[] linha 268
[] linha 434
[] linha 448
[] linha 543
[] linha 713


Tratando e produzindo colunas com informações das categorias:

(Ativar se for refazer.)

In [ ]:
# df_loc2 = tratando(df_loc, salva=True)
# df_loc2

Salvo


,fsq_id,categorias,nome,lat,lon,endereco,cidade,bairro,cat_nome,cat_id,cat_sup
0,4c30afa6452620a139721f0f,"[{'id': 13083, 'name': 'Churrascaria', 'icon':...",Churrascaria Pastoriza,-30.158138,-51.194270,"Avenida Juca Batista, 4023, Porto Alegre, RS, ...",Porto Alegre,NaN,Churrascaria,13083,Food
1,4d6a7586342b8cfad31cd72c,"[{'id': 13079, 'name': 'Brazilian Restaurant',...",Biskaia,-30.157387,-51.196786,"Avenida Juca Batista, 3782, Porto Alegre, RS, ...",Porto Alegre,NaN,Brazilian Restaurant,13079,Food
2,4e7e51be61af0051f2f60efe,"[{'id': 17069, 'name': 'Grocery Store / Superm...",J. Carboni Supermercado,-30.157197,-51.197027,"Avenida Juca Batista, 3821, Porto Alegre, RS, ...",Porto Alegre,Hípica,Grocery Store / Supermarket,17069,Residences
3,590257a9e185f3577bacba57,"[{'id': 13048, 'name': 'Pie Shop', 'icon': {'p...",Bella Gula,-30.158703,-51.193424,"Zaffari, Porto Alegre, RS, 91770-001",Porto Alegre,Hípica,Pie Shop,13048,Food
4,4f3a8595e4b01190396d76c1,"[{'id': 11045, 'name': 'Bank', 'icon': {'prefi...",CAIXA,-30.158300,-51.195022,"Rua Doutor Hermes Pacheco, 25 (Ag. Ag: 3481 - ...",Porto Alegre,NaN,Bank,11045,Colleges & Universities
...,...,...,...,...,...,...,...,...,...,...,...
835,595ce17df5e9d7161f191977,"[{'id': 17069, 'name': 'Grocery Store / Superm...",Mercado Muraro,-30.068700,-51.167293,"Rua Vinte e Seis de Dezembro, 245 (Vidal de ne...",Porto Alegre,NaN,Grocery Store / Supermarket,17069,Residences
836,4da87f41fa8c4175d0bb5320,"[{'id': 17110, 'name': 'Pet Supplies Store', '...",Aloha Pet Shop,-30.064177,-51.170898,"Rua Marechal Mallet, 401 (R. Saldanha da Gama)...",Porto Alegre,NaN,Pet Supplies Store,17110,Residences
837,50ce3a60e4b03ff34722b153,"[{'id': 16032, 'name': 'Park', 'icon': {'prefi...",Parque do Iberê Camargo,-30.070915,-51.169867,NaN,NaN,NaN,Park,16032,Professional & Other Places
838,5849eedcd8f3e9271e4ecdbd,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",Burger King,-30.061245,-51.163762,"Carrefour, Porto Alegre, RS",Porto Alegre,NaN,Fast Food Restaurant,13145,Food


Obtendo informações sobre bairros a partir das coordenadas de cada local:

(Ativar se for refazer.)

In [ ]:
# df_loc3 = obter_bairros(df_loc2, salva=True)
# df_loc3

Salvo


,fsq_id,categorias,nome,lat,lon,endereco,cidade,bairro,cat_nome,cat_id,cat_sup,bairro_geoloc
0,4c30afa6452620a139721f0f,"[{'id': 13083, 'name': 'Churrascaria', 'icon':...",Churrascaria Pastoriza,-30.158138,-51.194270,"Avenida Juca Batista, 4023, Porto Alegre, RS, ...",Porto Alegre,NaN,Churrascaria,13083,Food,Aberta dos Morros
1,4d6a7586342b8cfad31cd72c,"[{'id': 13079, 'name': 'Brazilian Restaurant',...",Biskaia,-30.157387,-51.196786,"Avenida Juca Batista, 3782, Porto Alegre, RS, ...",Porto Alegre,NaN,Brazilian Restaurant,13079,Food,Belém Novo
2,4e7e51be61af0051f2f60efe,"[{'id': 17069, 'name': 'Grocery Store / Superm...",J. Carboni Supermercado,-30.157197,-51.197027,"Avenida Juca Batista, 3821, Porto Alegre, RS, ...",Porto Alegre,Hípica,Grocery Store / Supermarket,17069,Residences,Belém Novo
3,590257a9e185f3577bacba57,"[{'id': 13048, 'name': 'Pie Shop', 'icon': {'p...",Bella Gula,-30.158703,-51.193424,"Zaffari, Porto Alegre, RS, 91770-001",Porto Alegre,Hípica,Pie Shop,13048,Food,Aberta dos Morros
4,4f3a8595e4b01190396d76c1,"[{'id': 11045, 'name': 'Bank', 'icon': {'prefi...",CAIXA,-30.158300,-51.195022,"Rua Doutor Hermes Pacheco, 25 (Ag. Ag: 3481 - ...",Porto Alegre,NaN,Bank,11045,Colleges & Universities,Aberta dos Morros
...,...,...,...,...,...,...,...,...,...,...,...,...
835,595ce17df5e9d7161f191977,"[{'id': 17069, 'name': 'Grocery Store / Superm...",Mercado Muraro,-30.068700,-51.167293,"Rua Vinte e Seis de Dezembro, 245 (Vidal de ne...",Porto Alegre,NaN,Grocery Store / Supermarket,17069,Residences,Vila São José
836,4da87f41fa8c4175d0bb5320,"[{'id': 17110, 'name': 'Pet Supplies Store', '...",Aloha Pet Shop,-30.064177,-51.170898,"Rua Marechal Mallet, 401 (R. Saldanha da Gama)...",Porto Alegre,NaN,Pet Supplies Store,17110,Residences,Vila João Pessoa
837,50ce3a60e4b03ff34722b153,"[{'id': 16032, 'name': 'Park', 'icon': {'prefi...",Parque do Iberê Camargo,-30.070915,-51.169867,NaN,NaN,NaN,Park,16032,Professional & Other Places,Vila São José
838,5849eedcd8f3e9271e4ecdbd,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",Burger King,-30.061245,-51.163762,"Carrefour, Porto Alegre, RS",Porto Alegre,NaN,Fast Food Restaurant,13145,Food,Partenon


Resetando `df_loc3` para não depender da requisição do geopy:

In [51]:
arq_csv = 'todas_amostras_tratadas_com_bairros.csv'
df_loc3 = pd.read_csv(caminho + arq_csv, index_col='Unnamed: 0')
df_loc3.drop('categorias', axis=1, inplace=True)
df_loc3

,fsq_id,nome,lat,lon,endereco,cidade,bairro,cat_nome,cat_id,cat_sup,bairro_geoloc
0,4c30afa6452620a139721f0f,Churrascaria Pastoriza,-30.158138,-51.194270,"Avenida Juca Batista, 4023, Porto Alegre, RS, ...",Porto Alegre,NaN,Churrascaria,13083,Food,Aberta dos Morros
1,4d6a7586342b8cfad31cd72c,Biskaia,-30.157387,-51.196786,"Avenida Juca Batista, 3782, Porto Alegre, RS, ...",Porto Alegre,NaN,Brazilian Restaurant,13079,Food,Belém Novo
2,4e7e51be61af0051f2f60efe,J. Carboni Supermercado,-30.157197,-51.197027,"Avenida Juca Batista, 3821, Porto Alegre, RS, ...",Porto Alegre,Hípica,Grocery Store / Supermarket,17069,Residences,Belém Novo
3,590257a9e185f3577bacba57,Bella Gula,-30.158703,-51.193424,"Zaffari, Porto Alegre, RS, 91770-001",Porto Alegre,Hípica,Pie Shop,13048,Food,Aberta dos Morros
4,4f3a8595e4b01190396d76c1,CAIXA,-30.158300,-51.195022,"Rua Doutor Hermes Pacheco, 25 (Ag. Ag: 3481 - ...",Porto Alegre,NaN,Bank,11045,Colleges & Universities,Aberta dos Morros
...,...,...,...,...,...,...,...,...,...,...,...
835,595ce17df5e9d7161f191977,Mercado Muraro,-30.068700,-51.167293,"Rua Vinte e Seis de Dezembro, 245 (Vidal de ne...",Porto Alegre,NaN,Grocery Store / Supermarket,17069,Residences,Vila São José
836,4da87f41fa8c4175d0bb5320,Aloha Pet Shop,-30.064177,-51.170898,"Rua Marechal Mallet, 401 (R. Saldanha da Gama)...",Porto Alegre,NaN,Pet Supplies Store,17110,Residences,Vila João Pessoa
837,50ce3a60e4b03ff34722b153,Parque do Iberê Camargo,-30.070915,-51.169867,NaN,NaN,NaN,Park,16032,Professional & Other Places,Vila São José
838,5849eedcd8f3e9271e4ecdbd,Burger King,-30.061245,-51.163762,"Carrefour, Porto Alegre, RS",Porto Alegre,NaN,Fast Food Restaurant,13145,Food,Partenon


Agora todas as linhas têm informações de bairro e catagoria:

In [52]:
df_loc3_subset = df_loc3[['cat_nome','cat_id','cat_sup','bairro_geoloc']]
for (col_nome, col_data) in df_loc3_subset.iteritems():
    print(f'Nulos em {col_nome}: ', col_data.isnull().sum())

Nulos em cat_nome:  0
Nulos em cat_id:  0
Nulos em cat_sup:  0
Nulos em bairro_geoloc:  0


Número de locais em cada bairro:

In [53]:
df_loc3['bairro_geoloc'].value_counts().to_frame().T

,Tristeza,Partenon,Belém Novo,Jardim Botânico,Ipanema,Glória,Menino Deus,Floresta,Praia de Belas,Boa Vista,Vila João Pessoa,Chácara das Pedras,Santana,Hípica,Vila Nova,Vila Ipiranga,Passo da Areia,Sarandi,Cristo Redentor,São Geraldo,Humaitá,Rio Branco,Santa Maria Goretti,Lami,Lomba do Pinheiro,Teresópolis,Jardim Leopoldina,Jardim Itu,Restinga,Santa Rosa de Lima,Santo Antônio,Moinhos de Vento,Montserrat,Independência,Cristal,Bom Fim,Cidade Baixa,Centro Histórico,Nonoai,Petrópolis,São João,Anchieta,Camaquã,Arquipélago,Belém Velho,Higienópolis,Auxiliadora,Bela Vista,Pedra Redonda,Jardim Lindóia,Santa Cecília,Vila Assunção,Campo Novo,Jardim Carvalho,Farrapos,Guarujá,Morro Santana,São Sebastião,Jardim do Salso,Boa Vista do Sul,Navegantes,Jardim São Pedro,Cavalhada,Costa e Silva,Azenha,Vila Jardim,Jardim Sabará,Vila Conceição,Três Figueiras,Farroupilha,Ponta Grossa,Mário Quintana,Pitinga,Aberta dos Morros,Jardim Isabel,Bom Jesus,Cascata,Medianeira,Lageado,Serraria,Extrema,Parque Santa Fé,Jardim Floresta,Santa Tereza,Espírito Santo,Coronel Aparício Borges,Jardim Europa,Passo das Pedras,Chapéu do Sol,São Tomé,Vila São José,São Caetano,Sétimo Céu,Rubem Berta
bairro_geoloc,23,20,17,16,16,15,15,15,14,13,13,13,13,13,12,12,12,11,11,11,11,11,11,11,11,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,9,9,9,9,9,9,9,9,9,9,9,9,9,9,8,8,8,8,8,7,7,7,7,7,7,7,7,7,6,6,6,6,6,5,5,5,5,5,4,4,3,2,2,2,2,2,2,1,1,1


In [54]:
print(f"Média de locais em cada bairro: {df_loc3['bairro_geoloc'].value_counts().mean()}")
print(f"Mediana de locais em cada bairro: {int(df_loc3['bairro_geoloc'].value_counts().median())}")

Média de locais em cada bairro: 8.936170212765957
Mediana de locais em cada bairro: 9


###1.4.3 Visualizando pontos dos lugares obtidos no mapa

Definindo função modificada para adicionar círculos pequenos indicando os locais e plotando o mapa:

In [55]:
def add_circ_mark2(my_map, my_data):
    for k,row in my_data.iterrows():
        coord = (row['lat'], row['lon'])
        bairro = row['bairro_geoloc']
        # raio = row['raio_fill']
        # marker = folium.Marker(coord, popup=bairro)
        circle = folium.Circle(coord, radius=50, color='#d35400', fill=True)
        my_map.add_child(circle) #.add_child(marker)
    return my_map
coord_poa = (-30.0324999, -51.2303767) 
enter_view = (coord_poa[0]-0.06, coord_poa[1]+0.05)
fig = folium.Figure(width=900, height=600)
map = folium.Map(location=enter_view, zoom_start=11).add_to(fig)
add_circ_mark2(map, df_loc3)

###1.4.4 Verificando as cidades dos locais obtidos

Temos linhas sem informação de cidade que podem ser de cidades vizinhas:

In [56]:
sem_cidade = df_loc3[df_loc3[['endereco','cidade']].isnull().any(1)]
display(sem_cidade.shape, sem_cidade)

(103, 11)

,fsq_id,nome,lat,lon,endereco,cidade,bairro,cat_nome,cat_id,cat_sup,bairro_geoloc
5,5dfa4199c83bc80008145531,Burger King,-30.158838,-51.193275,Rio Grande do Sul,NaN,NaN,Fast Food Restaurant,13145,Food,Aberta dos Morros
6,508161dfe4b01d45a112f0d5,Allier Estética,-30.159175,-51.188413,NaN,NaN,NaN,Health and Beauty Service,11061,Colleges & Universities,Hípica
7,505cd16ce4b08dae9ab357de,Praça do Meio,-30.155794,-51.197901,NaN,NaN,NaN,Plaza,16041,Professional & Other Places,Aberta dos Morros
19,500e9f8ce4b0294018e799ea,Mercado e Padaria Pão Quente,-30.057866,-51.160893,NaN,NaN,NaN,Candy Store,17060,Residences,Partenon
24,51e2c381498ebf0d29b6a82b,Ruazinha da Arena do Tricolor,-29.975343,-51.174692,NaN,NaN,NaN,Soccer Field,18064,Shops & Services,Anchieta
...,...,...,...,...,...,...,...,...,...,...,...
817,502fa402e4b029822643856d,Madeireira São João,-30.067242,-51.176114,SP,NaN,NaN,Hardware Store,17090,Residences,Vila João Pessoa
819,520e8cce11d24dd82cf66390,Neusa Nunes Hair,-30.067281,-51.176985,NaN,NaN,NaN,Health and Beauty Service,11061,Colleges & Universities,Vila João Pessoa
829,504bb10fe4b02e543906d4f1,Supermercado Caçula,-30.117847,-51.206669,GO,NaN,NaN,Storage Facility,11165,Colleges & Universities,Vila Nova
831,54bd4ad2498e5618f8421ac8,Villas Café,-30.118787,-51.207576,RS,NaN,NaN,Café,13034,Food,Vila Nova


Precisamos verificar se os bairros obtidos pelas coordenadas estão na lista de bairros de Porto Alegre. 

Os seguintes não estão, mas muitos é porque o nome do bairro está escrito diferente:

In [57]:
df_loc3[~df_loc3['bairro_geoloc'].isin(lista_bairros)]

,fsq_id,nome,lat,lon,endereco,cidade,bairro,cat_nome,cat_id,cat_sup,bairro_geoloc
216,505620f1e4b0c6b686ba8365,Marrom Glace,-30.015396,-51.118062,"Avenida Baltazar de Oliveira, Porto Alegre, RS",Porto Alegre,NaN,Dessert Shop,13040,Food,Jardim Leopoldina
219,4f313cdee4b0e6be3adff413,Natacha,-30.015501,-51.113470,"Avenida Baltazar de Oliveira Garcia, 3167, Por...",Porto Alegre,NaN,Brazilian Restaurant,13079,Food,Jardim Leopoldina
222,53ab57c8498ea40e46a5c0db,Rafa Sushi,-30.015501,-51.110116,"Avenida Baltazar de Oliveira Garcia, 3422, Por...",Porto Alegre,NaN,Buffet,13030,Food,Jardim Leopoldina
225,4f73b60ce4b0f3ab57f359f4,Rancho do Xis,-30.015501,-51.110116,"Avenida Baltazar de Oliveira Garcia, 3398, Por...",Porto Alegre,NaN,Brazilian Restaurant,13079,Food,Jardim Leopoldina
308,4f512f38e4b07e4c614b92f3,Restaurante do Vanderlei,-30.010766,-51.181651,"Rua Coronel Feijó, 289, Porto Alegre, RS",Porto Alegre,NaN,Brazilian Restaurant,13079,Food,Passo da Areia
315,4d3de53ad2c8f04d9d2f6e72,Suzana d'Ávila Studio de Dança,-30.009193,-51.179166,"Rua General João de Deus Martins, 148, Porto A...",Porto Alegre,NaN,Dance Studio,18025,Shops & Services,Passo da Areia
374,4e9801a193adc6fb83596bf6,Praça México,-30.019682,-51.110595,"Avenida Juscelino Kubitschek de Oliveira, S/N,...",Porto Alegre,NaN,Plaza,16041,Professional & Other Places,Jardim Leopoldina
375,501594b9e4b09cb7e0494d24,Padaria e Confeitaria Colonial,-30.015775,-51.113916,"Rua Doutor Carlos Maria Bins, 3031 (Av. Baltaz...",Porto Alegre,NaN,Bakery,13002,Food,Jardim Leopoldina
376,56f31496498e5b8ed52374a9,Feira Modelo Leopoldina,-30.017843,-51.114247,NaN,NaN,NaN,Food Stand,13053,Food,Jardim Leopoldina
377,56314028498e09b260ad35e9,Farmácia Leopoldina,-30.015731,-51.110746,"Avenida Baltazar de Oliveira Garcia, 3305, Por...",Porto Alegre,NaN,Drugstore,17035,Residences,Jardim Leopoldina


In [58]:
print(lista_bairros)

['Aberta dos Morros', 'Agronomia', 'Anchieta', 'Arquipélago', 'Auxiliadora', 'Azenha', 'Bela Vista', 'Belém Novo', 'Belém Velho', 'Boa Vista', 'Boa Vista do Sul', 'Bom Fim', 'Bom Jesus', 'Camaquã', 'Campo Novo', 'Cascata', 'Cavalhada', 'Centro Histórico', 'Chapéu do Sol', 'Chácara das Pedras', 'Cidade Baixa', 'Coronel Aparício Borges', 'Costa e Silva', 'Cristal', 'Cristo Redentor', 'Espírito Santo', 'Extrema', 'Farrapos', 'Farroupilha', 'Floresta', 'Glória', 'Guarujá', 'Higienópolis', 'Humaitá', 'Hípica', 'Independência', 'Ipanema', 'Jardim Botânico', 'Jardim Carvalho', 'Jardim Dona Leopoldina', 'Jardim Floresta', 'Jardim Isabel', 'Jardim Itu', 'Jardim Itu-Sabará', 'Jardim Lindóia', 'Jardim Sabará', 'Jardim São Pedro', 'Jardim do Salso', 'Lageado', 'Lami', 'Lomba do Pinheiro', 'Marcílio Dias', 'Medianeira', 'Menino Deus', 'Moinhos de Vento', "Mont'Serrat", 'Morro Santana', 'Mário Quintana', 'Navegantes', 'Nonoai', 'Parque Santa Fé', 'Partenon', "Passo D'Areia", 'Passo das Pedras', 'Ped

Isso será corrigido agora:

In [59]:
dict_corrige = {'Passo da Areia': "Passo D'Areia",
                'Montserrat': "Mont'Serrat",
                'Jardim Leopoldina': 'Jardim Dona Leopoldina'}
df_loc3['bairro_geoloc'] = df_loc3['bairro_geoloc'].map(dict_corrige).fillna(df_loc3['bairro_geoloc'])

Conferindo:

In [60]:
df_loc3[~df_loc3['bairro_geoloc'].isin(lista_bairros)]

,fsq_id,nome,lat,lon,endereco,cidade,bairro,cat_nome,cat_id,cat_sup,bairro_geoloc
602,4f305094e4b0118a88c73b5b,Panvel Bem Estar,-30.134255,-51.095095,"Avenida 24 de Outubro, 722, Porto Alegre, RS",Porto Alegre,NaN,Drugstore,17035,Residences,São Tomé
603,51e973a0498ebf4bb2b5328b,Teg Modas,-30.134030,-51.093408,"Lomba do Pinheiro, Porto Alegre, RS",Porto Alegre,NaN,Women's Store,17052,Residences,São Tomé
801,5011990816483ceb8a598a02,Salon Bardot,-30.022477,-51.155176,"Avenida Tulio de Rose 580 /, 107, Porto Alegre...",Porto Alegre,NaN,Hair Salon,11064,Colleges & Universities,Jardim Europa
814,4f871615e4b09efb9f5a4734,Savarauto,-30.029260,-51.157003,"Avenida Nilo Peçanha, 3410, Porto Alegre, RS",Porto Alegre,NaN,Car Dealership,17006,Residences,Jardim Europa


Dos que sobraram, São Tomé é um bairro de Viamão, e portanto será descartado:

In [61]:
df_loc3 = df_loc3[df_loc3['bairro_geoloc']!='São Tomé'].reset_index().drop('index',axis=1)

In [62]:
df_loc3

,fsq_id,nome,lat,lon,endereco,cidade,bairro,cat_nome,cat_id,cat_sup,bairro_geoloc
0,4c30afa6452620a139721f0f,Churrascaria Pastoriza,-30.158138,-51.194270,"Avenida Juca Batista, 4023, Porto Alegre, RS, ...",Porto Alegre,NaN,Churrascaria,13083,Food,Aberta dos Morros
1,4d6a7586342b8cfad31cd72c,Biskaia,-30.157387,-51.196786,"Avenida Juca Batista, 3782, Porto Alegre, RS, ...",Porto Alegre,NaN,Brazilian Restaurant,13079,Food,Belém Novo
2,4e7e51be61af0051f2f60efe,J. Carboni Supermercado,-30.157197,-51.197027,"Avenida Juca Batista, 3821, Porto Alegre, RS, ...",Porto Alegre,Hípica,Grocery Store / Supermarket,17069,Residences,Belém Novo
3,590257a9e185f3577bacba57,Bella Gula,-30.158703,-51.193424,"Zaffari, Porto Alegre, RS, 91770-001",Porto Alegre,Hípica,Pie Shop,13048,Food,Aberta dos Morros
4,4f3a8595e4b01190396d76c1,CAIXA,-30.158300,-51.195022,"Rua Doutor Hermes Pacheco, 25 (Ag. Ag: 3481 - ...",Porto Alegre,NaN,Bank,11045,Colleges & Universities,Aberta dos Morros
...,...,...,...,...,...,...,...,...,...,...,...
833,595ce17df5e9d7161f191977,Mercado Muraro,-30.068700,-51.167293,"Rua Vinte e Seis de Dezembro, 245 (Vidal de ne...",Porto Alegre,NaN,Grocery Store / Supermarket,17069,Residences,Vila São José
834,4da87f41fa8c4175d0bb5320,Aloha Pet Shop,-30.064177,-51.170898,"Rua Marechal Mallet, 401 (R. Saldanha da Gama)...",Porto Alegre,NaN,Pet Supplies Store,17110,Residences,Vila João Pessoa
835,50ce3a60e4b03ff34722b153,Parque do Iberê Camargo,-30.070915,-51.169867,NaN,NaN,NaN,Park,16032,Professional & Other Places,Vila São José
836,5849eedcd8f3e9271e4ecdbd,Burger King,-30.061245,-51.163762,"Carrefour, Porto Alegre, RS",Porto Alegre,NaN,Fast Food Restaurant,13145,Food,Partenon


Enquanto isso, 

* Jardim Europa é um bairro de Porto Alegre que por algum motivo não estava na lista encontrada na página da Wikipédia. 

* Ele não corresponde à Zona Indefinida que havia sido descartada porque esta tem uma área muito grande, com mais de 10 mil hectares, enquanto o Jardim Europa é um bairro bem pequeno.

* Este bairro será adicionado *a posteriori* no dataframe dos bairros com coordenadas obtidas com o `geopy`, pois o geocoder com arcgis fornece um local em Mato Grosso do Sul.

### 1.4.5 Adicionando bairro que não constava na lista

In [63]:
def geopy(bairro):
    geolocator = Nominatim(user_agent='TestBot')
    g = geolocator.geocode(f"{bairro}, Porto Alegre, Brasil")
    return (g.latitude, g.longitude)

In [64]:
dict_jardim_europa = {'bairro': 'Jardim Europa',
                      'area': np.nan,
                      'pop2010': np.nan,
                      'densidade': np.nan,
                      'renda': np.nan,
                      'raio': np.nan,
                      'lat': geopy('Jardim Europa')[0],
                      'lon': geopy('Jardim Europa')[1],
                      'raio_fill': 1000}
dict_jardim_europa

{'area': nan,
 'bairro': 'Jardim Europa',
 'densidade': nan,
 'lat': -30.0245527,
 'lon': -51.158451,
 'pop2010': nan,
 'raio': nan,
 'raio_fill': 1000,
 'renda': nan}

In [65]:
# df = df[0:95]
df = df.append(dict_jardim_europa, ignore_index=True)
# df.to_csv(caminho + 'tabela2.csv')
df

,bairro,area,pop2010,densidade,renda,raio,lat,lon,raio_fill
0,Aberta dos Morros,NaN,NaN,NaN,NaN,NaN,-30.160022,-51.197486,1560
1,Agronomia,12410000.0,12222.0,"8,6 hab/ha","3,98 SM/mês",1987.517468,-30.069267,-51.149217,2981
2,Anchieta,840000.0,203.0,"2,4 hab/ha","8,41 SM/mês",517.088295,-29.975652,-51.174903,775
3,Arquipélago,47180000.0,5061.0,"1,1 hab/ha","2,96 SM/mês",1040.130159,-29.992760,-51.226618,1560
4,Auxiliadora,820000.0,9985.0,"121,8 hab/ha","19,57 SM/mês",510.895397,-30.020011,-51.190588,766
...,...,...,...,...,...,...,...,...,...
91,Vila Jardim,1780000.0,14251.0,"80,1 hab/ha","5,30 SM/mês",752.722789,-30.035177,-51.153283,1129
92,Vila João Pessoa,840000.0,10522.0,"125,3 hab/ha","5,58 SM/mês",517.088295,-30.065375,-51.176165,775
93,Vila Nova,10310000.0,33145.0,"32,1 hab/ha","5,35 SM/mês",1811.566981,-30.116003,-51.207517,2717
94,Vila São José,2120000.0,28957.0,"136,6 hab/ha","3,60 SM/mês",821.472433,-30.068057,-51.166311,1232


Ordenando `df` na ordem alfabética dos bairros e resetando o índice:

In [70]:
df = df.sort_values('bairro').reset_index().drop('index',axis=1)
df

,bairro,area,pop2010,densidade,renda,raio,lat,lon,raio_fill
0,Aberta dos Morros,NaN,NaN,NaN,NaN,NaN,-30.160022,-51.197486,1560
1,Agronomia,12410000.0,12222.0,"8,6 hab/ha","3,98 SM/mês",1987.517468,-30.069267,-51.149217,2981
2,Anchieta,840000.0,203.0,"2,4 hab/ha","8,41 SM/mês",517.088295,-29.975652,-51.174903,775
3,Arquipélago,47180000.0,5061.0,"1,1 hab/ha","2,96 SM/mês",1040.130159,-29.992760,-51.226618,1560
4,Auxiliadora,820000.0,9985.0,"121,8 hab/ha","19,57 SM/mês",510.895397,-30.020011,-51.190588,766
...,...,...,...,...,...,...,...,...,...
91,Vila Ipiranga,5200000.0,20951.0,"40,3 hab/ha","8,99 SM/mês",1286.550197,-30.022920,-51.153680,1929
92,Vila Jardim,1780000.0,14251.0,"80,1 hab/ha","5,30 SM/mês",752.722789,-30.035177,-51.153283,1129
93,Vila João Pessoa,840000.0,10522.0,"125,3 hab/ha","5,58 SM/mês",517.088295,-30.065375,-51.176165,775
94,Vila Nova,10310000.0,33145.0,"32,1 hab/ha","5,35 SM/mês",1811.566981,-30.116003,-51.207517,2717


Visualizando no mapa:

In [71]:
def add_circ_mark3(my_map, my_data):
    for k,row in my_data.iterrows():
        coord = (row['lat'], row['lon'])
        bairro = row['bairro']
        raio = row['raio_fill']
        marker = folium.Marker(coord, popup=bairro)
        if row['bairro']=='Jardim Europa':
            circle = folium.Circle(coord, radius=raio, color='#d35400', fill=True)
        else:
            circle = folium.Circle(coord, radius=50, color='#d35400', fill=True)
        my_map.add_child(circle).add_child(marker)
    return my_map

enter_view = geopy('Jardim Europa')
fig = folium.Figure(width=900, height=600)
map = folium.Map(location=enter_view, zoom_start=15).add_to(fig)
add_circ_mark3(map, df)

Verificando nomes de bairros em `df_loc3` que não estejam em `df`:

In [79]:
df_loc3[~df_loc3['bairro_geoloc'].isin(df['bairro'])] # (ok)

,fsq_id,nome,lat,lon,endereco,cidade,bairro,cat_nome,cat_id,cat_sup,bairro_geoloc


Verificando nomes de bairros em `df` que não estejam em `df_loc3`:

In [78]:
df[~df['bairro'].isin(df_loc3['bairro_geoloc'])]

,bairro,area,pop2010,densidade,renda,raio,lat,lon,raio_fill
1,Agronomia,12410000.0,12222.0,"8,6 hab/ha","3,98 SM/mês",1987.517468,-30.069267,-51.149217,2981
44,Jardim Itu-Sabará,4570000.0,31127.0,"68,1 hab/ha","9,17 SM/mês",1206.099573,-30.033103,-51.145110,1809
52,Marcílio Dias,1240000.0,598.0,"4,8 hab/ha","1,51 SM/mês",628.254931,-30.016294,-51.215418,942


Provavelmente não foram encontrados locais nesses 3 bairros em razão da sobreposição com outros bairros, resultando na exclusão das linhas duplicadas.

Salvando em CSV:

In [86]:
# df.to_csv(caminho + 'tabela2.csv')

##1.5 Preparando dataframe para clusterização

Definindo colunas importantes no dataframe dos locais para aplicar a clusterização:

In [80]:
df_loc3_subset = df_loc3[['cat_nome','cat_id','cat_sup','bairro_geoloc']]

In [81]:
df_loc3_subset['bairro_geoloc'].unique()

array(['Aberta dos Morros', 'Belém Novo', 'Hípica', 'Partenon',
       'Jardim Carvalho', 'Anchieta', 'Arquipélago', 'Auxiliadora',
       'Higienópolis', 'Azenha', 'Menino Deus', 'Bela Vista',
       'Petrópolis', 'Belém Velho', 'Vila Nova', 'Boa Vista',
       'Boa Vista do Sul', 'Lageado', 'Bom Fim', 'Chácara das Pedras',
       'Bom Jesus', 'Camaquã', 'Tristeza', 'Campo Novo', 'Cascata',
       'Glória', 'Coronel Aparício Borges', 'Nonoai', 'Cavalhada',
       'Centro Histórico', 'Chapéu do Sol', 'Ponta Grossa',
       'Cidade Baixa', 'Vila João Pessoa', 'Jardim Dona Leopoldina',
       'Costa e Silva', 'Cristal', 'Cristo Redentor', 'Vila Ipiranga',
       'Ipanema', 'Espírito Santo', 'Guarujá', 'Extrema', 'Farrapos',
       'Farroupilha', 'Santana', 'Floresta', 'São Geraldo', 'Teresópolis',
       'Serraria', "Passo D'Areia", 'Humaitá', 'Independência',
       'Jardim Botânico', 'Jardim Floresta', 'Jardim Lindóia',
       'Jardim São Pedro', 'Jardim Isabel', 'Pedra Redonda', 'Jard

Gerando colunas dummies e concatenando no dataframe:

In [82]:
df_locais = df_loc3_subset.copy()
dummies = pd.get_dummies(df_locais['cat_sup'])
df_locais = pd.concat([df_locais, dummies],axis=1)
display(df_locais.shape, df_locais)

(838, 13)

,cat_nome,cat_id,cat_sup,bairro_geoloc,Arts & Entertainment,Colleges & Universities,Events,Food,Outdoors & Recreation,Professional & Other Places,Residences,Shops & Services,Travel & Transport
0,Churrascaria,13083,Food,Aberta dos Morros,0,0,0,1,0,0,0,0,0
1,Brazilian Restaurant,13079,Food,Belém Novo,0,0,0,1,0,0,0,0,0
2,Grocery Store / Supermarket,17069,Residences,Belém Novo,0,0,0,0,0,0,1,0,0
3,Pie Shop,13048,Food,Aberta dos Morros,0,0,0,1,0,0,0,0,0
4,Bank,11045,Colleges & Universities,Aberta dos Morros,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,Grocery Store / Supermarket,17069,Residences,Vila São José,0,0,0,0,0,0,1,0,0
834,Pet Supplies Store,17110,Residences,Vila João Pessoa,0,0,0,0,0,0,1,0,0
835,Park,16032,Professional & Other Places,Vila São José,0,0,0,0,0,1,0,0,0
836,Fast Food Restaurant,13145,Food,Partenon,0,0,0,1,0,0,0,0,0


Agrupando bairros e contando quantos locais há de cada categoria:

In [84]:
df_group = df_locais[dummies.columns.tolist()+['bairro_geoloc']]
df_group = df_group.groupby('bairro_geoloc').sum()
# df_group.to_csv(caminho + 'categorias_por_bairro.csv')
df_group

,Arts & Entertainment,Colleges & Universities,Events,Food,Outdoors & Recreation,Professional & Other Places,Residences,Shops & Services,Travel & Transport
bairro_geoloc,,,,,,,,,
Aberta dos Morros,0,1,0,3,0,1,1,0,0
Anchieta,0,0,0,6,0,0,3,1,0
Arquipélago,0,1,0,1,0,7,1,0,0
Auxiliadora,0,0,0,8,0,0,1,0,0
Azenha,0,2,0,3,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...
Vila Ipiranga,0,2,0,7,0,2,1,0,0
Vila Jardim,0,3,0,2,0,0,2,0,0
Vila João Pessoa,2,1,0,4,0,1,5,0,0


In [85]:
df_group.sum(axis=0)

Arts & Entertainment            50
Colleges & Universities        130
Events                           4
Food                           342
Outdoors & Recreation            1
Professional & Other Places     86
Residences                     184
Shops & Services                39
Travel & Transport               2
dtype: int64

Podemos perceber que não foi obtido nenhum local classificado como `'Nightlife Spots'`, o que é bastante estranho, considerando que se trata de uma capital.

Num segundo momento serão buscados mais dados pela API.

#2 Rascunho (ignorar)

In [159]:
dict_cat

{10: 'Arts & Entertainment',
 11: 'Colleges & Universities',
 12: 'Events',
 13: 'Food',
 14: 'Nightlife Spots',
 15: 'Outdoors & Recreation',
 16: 'Professional & Other Places',
 17: 'Residences',
 18: 'Shops & Services',
 19: 'Travel & Transport'}

In [184]:
limites_sup = [
    57,
    180,
    117,
    389,
    16,
    57,
    55,
    140,
    77,
    57,
]
lista_de_arranjos = []
for k,j in enumerate(limites_sup):
    lista_de_arranjos.append(np.arange(0,i)+(k+10)*1000)

In [187]:
dict_cat_inv = dict(zip(lista_cat_sup,lista_de_arranjos))
dict_cat_inv

{'Arts & Entertainment': array([10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008,
        10009, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10017,
        10018, 10019, 10020, 10021, 10022, 10023, 10024, 10025, 10026,
        10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035,
        10036, 10037, 10038, 10039, 10040, 10041, 10042, 10043, 10044,
        10045, 10046, 10047, 10048, 10049, 10050, 10051, 10052, 10053,
        10054, 10055, 10056]),
 'Colleges & Universities': array([11000, 11001, 11002, 11003, 11004, 11005, 11006, 11007, 11008,
        11009, 11010, 11011, 11012, 11013, 11014, 11015, 11016, 11017,
        11018, 11019, 11020, 11021, 11022, 11023, 11024, 11025, 11026,
        11027, 11028, 11029, 11030, 11031, 11032, 11033, 11034, 11035,
        11036, 11037, 11038, 11039, 11040, 11041, 11042, 11043, 11044,
        11045, 11046, 11047, 11048, 11049, 11050, 11051, 11052, 11053,
        11054, 11055, 11056]),
 'Events': array([12000, 12001, 120